In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

spot = pd.read_csv("ds4420_spotify.csv")
spot.head()

,song_title,artist_name,release_month,release_year,artist_pop,track_pop,duration_s,explicit
0,double take,Dhruv,1,2022,66,78,171.743,False
1,One Last Kiss,Hikaru Utada,3,2021,73,63,252.026,False
2,Autumn,NIKI,8,2022,79,72,232.800,False
3,Sweet Heat Lightning,Gregory Alan Isakov,8,2023,73,78,286.120,False
4,Brothers In Arms - Remastered 1996,Dire Straits,5,1985,81,74,420.240,False


In [2]:
spot.tail()

,song_title,artist_name,release_month,release_year,artist_pop,track_pop,duration_s,explicit
178,Finally Over It,Summer Walker,11,2025,80,70,143.746,False
179,Like a Tattoo,Sade,10,1992,81,82,218.000,False
180,Call On Me,Daniel Caesar,10,2025,88,70,169.004,False
181,Center Mass,Twenty One Pilots,9,2025,85,68,228.173,False
182,Mr. Brightside,The Killers,6,2004,80,91,222.973,False


In [3]:
# Scale the data
num_spot = spot.iloc[:, [4, 6]].to_numpy()
scaler = StandardScaler()
scale_spot = scaler.fit_transform(num_spot)
scale_spot[0:5,:]

array([[-0.5017626 , -0.72739308],
       [-0.08110953,  0.04624362],
       [ 0.27945024, -0.13902523],
       [-0.08110953,  0.37478603],
       [ 0.39963683,  1.667216  ]])

In [4]:
# Example x, y, z
x = scale_spot[0].reshape(-1, 1) # double take
y = scale_spot[1].reshape(-1, 1) # One Last Kiss
z = scale_spot[182].reshape(-1, 1) # Mr. Brightside
z

array([[ 0.33954354],
       [-0.23372184]])

In [5]:
# Calculate distances
# Notice R and Python have slightly different norm functions due to rounding
distances = {
    "L1_x_z": np.linalg.norm(z - x, ord=1),
    "L1_y_z": np.linalg.norm(z - y, ord=1),
    "L2_x_z": np.linalg.norm(z - x, ord=2),
    "L2_y_z": np.linalg.norm(z - y, ord=2),
    "Linf_x_z": np.linalg.norm(z - x, ord=np.inf),
    "Linf_y_z": np.linalg.norm(z - y, ord=np.inf),
}
distances

{'L1_x_z': np.float64(1.3349773802070826),
 'L1_y_z': np.float64(0.7006185308437332),
 'L2_x_z': np.float64(0.9754523622981865),
 'L2_y_z': np.float64(0.5053015573481016),
 'Linf_x_z': np.float64(0.8413061349714686),
 'Linf_y_z': np.float64(0.4206530674857343)}

In [6]:
# Cosine similarity
# individual pair (z vs. x)
print(f'S_C(z,x): {cosine_similarity(np.vstack([z.T, x.T]))[0, 1]}')

# all pairs
cosine_similarity(np.vstack([z.T, x.T, y.T]))

S_C(z,x): -0.0009954513839126454


array([[ 1.00000000e+00, -9.95451384e-04, -9.96416306e-01],
       [-9.95451384e-04,  1.00000000e+00,  8.55763838e-02],
       [-9.96416306e-01,  8.55763838e-02,  1.00000000e+00]])

In [7]:
# Collaborative Filtering Example
drg = np.array([[-5 / 3, 0, 4 / 3, 1 / 3]])
st1 = np.array([[1, -1, 0, 0]])
st2 = np.array([[-2, 2, 0, 0]])
st3 = np.array([[0, 1 / 3, 4 / 3, -5 / 3]])
st4 = np.array([[-2 / 3, 1 / 3, 0, 1 / 3]])

similarity_matrix = cosine_similarity(np.vstack((drg, st1, st2, st3, st4)))
similarity_matrix

array([[ 1.        , -0.54554473,  0.54554473,  0.26190476,  0.69293487],
       [-0.54554473,  1.        , -1.        , -0.10910895, -0.8660254 ],
       [ 0.54554473, -1.        ,  1.        ,  0.10910895,  0.8660254 ],
       [ 0.26190476, -0.10910895,  0.10910895,  1.        , -0.25197632],
       [ 0.69293487, -0.8660254 ,  0.8660254 , -0.25197632,  1.        ]])

In [8]:
predicted_rating = (
    similarity_matrix[0, 2] * 4 + similarity_matrix[0, 4] * 5
) / (similarity_matrix[0, 2] + similarity_matrix[0, 4])
predicted_rating

np.float64(4.559504469211125)

In [9]:
# MinMax Scaling of each Score
# This step is necessary!
# Set diagonal elements to NaN
np.fill_diagonal(similarity_matrix, np.nan)

sim_scores_scaled = pd.DataFrame(similarity_matrix).apply(
    lambda x: (x - np.nanmin(x)) / (np.nanmax(x) - np.nanmin(x)), axis=0
)

# NOTE: We look down the columnns! (the matrix isn't symmetric...)
print(sim_scores_scaled)

          0         1         2         3         4
0       NaN  0.510113  0.828255  1.000000  0.900066
1  0.000000       NaN  0.000000  0.278016  0.000000
2  0.880991  0.000000       NaN  0.702663  1.000000
3  0.651968  1.000000  0.594370       NaN  0.354521
4  1.000000  0.150383  1.000000  0.000000       NaN


In [10]:
(.510*4 + 1*1)/(.510 + 1)

2.013245033112583

In [11]:
# And, Dr. Gerber's Song 2 (for completion's sake) using all other users
print(sum(sim_scores_scaled.iloc[1:,0] * [3, 5, 3, 4]) / sum(sim_scores_scaled.iloc[1:,0]))

# Or, just the top 2
sum(sim_scores_scaled.iloc[[2,4],0] * [5, 4]) / sum(sim_scores_scaled.iloc[[2,4],0])

4.0904170507256365


4.468365363118093

In [12]:
# content-based filtering
# add the "explicit" column to the scaled data
explicit = spot['explicit'].astype(int).to_numpy().reshape(-1, 1)
full_spot = np.hstack((scale_spot, explicit))
full_spot[0:5,:]

array([[-0.5017626 , -0.72739308,  0.        ],
       [-0.08110953,  0.04624362,  0.        ],
       [ 0.27945024, -0.13902523,  0.        ],
       [-0.08110953,  0.37478603,  0.        ],
       [ 0.39963683,  1.667216  ,  0.        ]])

In [13]:
# Song #182 corresponds to "Mr. Brightside"
drg = full_spot[182].reshape(1, -1)
cosine_sim_to_drg = []

# calculate cosine similarity for each song relative to "Mr. Brightside"
for i in range(full_spot.shape[0]):
    temp_cosine = cosine_similarity(drg, full_spot[i].reshape(1, -1))[0, 0]
    cosine_sim_to_drg.append(temp_cosine)

similarity_df = pd.DataFrame({
    'song': spot['song_title'],
    'artist': spot['artist_name'],
    'sim_scores': cosine_sim_to_drg
})

# sort the DataFrame by similarity scores in descending order
similarity_df = similarity_df.sort_values(by='sim_scores', ascending=False)

similarity_df.head()

,song,artist,sim_scores
182,Mr. Brightside,The Killers,1.000000
179,Like a Tattoo,Sade,0.999939
175,WHERE IS MY HUSBAND!,RAYE,0.998915
128,We Don't Talk Anymore (feat. Selena Gomez),Charlie Puth,0.994790
57,How Long,Charlie Puth,0.994257


In [14]:
# Item-Item Example
item_mat = np.array([
    [2, np.nan, 5, 4],
    [5, 3, np.nan, 4],
    [1, 5, 3, np.nan],
    [np.nan, 3, 4, 1],
    [3, 4, np.nan, 4]
])
item_mat_scaled = item_mat - np.nanmean(item_mat, axis=0)
item_mat_scaled

array([[-0.75,   nan,  1.  ,  0.75],
       [ 2.25, -0.75,   nan,  0.75],
       [-1.75,  1.25, -1.  ,   nan],
       [  nan, -0.75,  0.  , -2.25],
       [ 0.25,  0.25,   nan,  0.75]])

In [15]:
# Compute pairwise cosine similarities for items
sim_scores = []
for i in range(item_mat_scaled.shape[1] - 1):
    for j in range(i + 1, item_mat_scaled.shape[1]):
        SongA = item_mat_scaled[:, i]
        SongB = item_mat_scaled[:, j]
        shared = ~np.isnan(SongA) & ~np.isnan(SongB)
        sim = cosine_similarity(SongA[shared].reshape(1, -1), SongB[shared].reshape(1, -1))[0, 0]
        sim_scores.append(sim)

sim_scores

[np.float64(-0.9008659350499821),
 np.float64(0.37139067635410367),
 np.float64(0.4236592728681617),
 np.float64(-0.8574929257125441),
 np.float64(0.4842001247062522),
 np.float64(0.31622776601683794)]

In [16]:
# You should be able to create a similarity matrix as in user-user
# But I will leave that to you on HW3
# Min-Max Scale for Song 2
Song2_scores = np.array([sim_scores[0], sim_scores[3], sim_scores[4]])
scale_Song2 = (Song2_scores - np.min(Song2_scores)) / (np.max(Song2_scores) - np.min(Song2_scores))
scale_Song2

array([0.        , 0.03131476, 1.        ])

In [17]:
# Weighted Average to Predict
sum(item_mat[0,~np.isnan(item_mat[0,])] * scale_Song2) / sum(scale_Song2)

np.float64(4.030363919803004)

In [18]:
import numpy as np

# Intuition behind SVD for Recommender Systems
# use the last matrix but centering by item and fill in the missing values
# (Note: I'm centering on item because that's what I did last; user means are more common)

R = np.array(item_mat_scaled, copy=True)
R[np.isnan(R)] = 0
R

array([[-0.75,  0.  ,  1.  ,  0.75],
       [ 2.25, -0.75,  0.  ,  0.75],
       [-1.75,  1.25, -1.  ,  0.  ],
       [ 0.  , -0.75,  0.  , -2.25],
       [ 0.25,  0.25,  0.  ,  0.75]])

In [19]:
# review the user-user matrix
user_user = R @ R.T
print(user_user)

[[ 2.125  -1.125   0.3125 -1.6875  0.375 ]
 [-1.125   6.1875 -4.875  -1.125   0.9375]
 [ 0.3125 -4.875   5.625  -0.9375 -0.125 ]
 [-1.6875 -1.125  -0.9375  5.625  -1.875 ]
 [ 0.375   0.9375 -0.125  -1.875   0.6875]]


In [20]:
# what are the eigenvectors
eigvals, eigvecs = np.linalg.eigh(user_user)  # eigh is for symmetric matrices
# remember those
print(eigvecs)

[[ 0.4472136  -0.24609875 -0.79876071  0.29964111 -0.10785145]
 [ 0.4472136  -0.38452974  0.31910844  0.13023406  0.73030526]
 [ 0.4472136  -0.28800159  0.49139502  0.16489873 -0.6696226 ]
 [ 0.4472136   0.08070788 -0.10234188 -0.88440684 -0.0289295 ]
 [ 0.4472136   0.8379222   0.09059913  0.28963294  0.0760983 ]]


In [21]:
# now, lets get the SVD
# In numpy, svd returns U, singular values s, and Vt
U, s, Vt = np.linalg.svd(R, full_matrices=False)

# R_full$u # should look a little familiar...
print(U)

[[-0.10785145 -0.29964111  0.79876071 -0.24609875]
 [ 0.73030526 -0.13023406 -0.31910844 -0.38452974]
 [-0.6696226  -0.16489873 -0.49139502 -0.28800159]
 [-0.0289295   0.88440684  0.10234188  0.08070788]
 [ 0.0760983  -0.28963294 -0.09059913  0.8379222 ]]


In [22]:
# Note: numpy gives Vt directly (V transpose)
V = Vt.T
print(V)

[[ 0.88025608  0.055295   -0.32924191  0.33718166]
 [-0.40587305 -0.31567115 -0.32549887  0.79351703]
 [ 0.16964416 -0.05038645  0.88536313  0.42990022]
 [ 0.1778684  -0.94591511  0.04221824 -0.26800198]]


In [23]:
# the S matrix is diagonal, python (like R) just gives those diagonal terms
print(s)

[3.31146768 2.67417917 1.45720518 0.09747109]


In [24]:
# Should be able to recover full matrix with
R_recovered = U @ np.diag(s) @ Vt
print(R_recovered)
print(R)

[[-7.50000000e-01 -2.97314216e-16  1.00000000e+00  7.50000000e-01]
 [ 2.25000000e+00 -7.50000000e-01 -4.96171197e-17  7.50000000e-01]
 [-1.75000000e+00  1.25000000e+00 -1.00000000e+00  5.11975673e-16]
 [-1.51953224e-16 -7.50000000e-01  3.03102151e-16 -2.25000000e+00]
 [ 2.50000000e-01  2.50000000e-01 -1.87705388e-16  7.50000000e-01]]
[[-0.75  0.    1.    0.75]
 [ 2.25 -0.75  0.    0.75]
 [-1.75  1.25 -1.    0.  ]
 [ 0.   -0.75  0.   -2.25]
 [ 0.25  0.25  0.    0.75]]


In [25]:
# But, the point is to pick only the most important latent features, k < 4
# Note, the diagonal terms are sorted; the matrices are already sorted so that
# the first k columns/rows are the most important! The fourth is much smaller than the third, so
k = 3
U_tilde = U[:, :k]
S_tilde = np.diag(s[:k])
VT_tilde = Vt[:k, :]

# Our prediction matrix
R_hat = U_tilde @ S_tilde @ VT_tilde
print(R_hat)

[[-0.74191185  0.0190345   1.01031224  0.7435713 ]
 [ 2.26263775 -0.72025856  0.01611289  0.73995514]
 [-1.74053469  1.27227547 -0.98793192 -0.00752331]
 [-0.0026525  -0.75624235 -0.00338189 -2.24789171]
 [ 0.2224613   0.18519094 -0.03511132  0.77188858]]


In [26]:
# Add the mean back to put it on the original scale
# I'm using item means because I'm lazy and that's how I created item_mat_scaled
# but, could have originally scaled with user means and done it that way (would give slightly different results)
item_means = np.nanmean(item_mat, axis=0)  # mean per item/column ignoring NaNs
R_hat_original_scale = R_hat + item_means
print(R_hat_original_scale)

# compare to original
print(item_mat)

[[2.00808815 3.7690345  5.01031224 3.9935713 ]
 [5.01263775 3.02974144 4.01611289 3.98995514]
 [1.00946531 5.02227547 3.01206808 3.24247669]
 [2.7473475  2.99375765 3.99661811 1.00210829]
 [2.9724613  3.93519094 3.96488868 4.02188858]]
[[ 2. nan  5.  4.]
 [ 5.  3. nan  4.]
 [ 1.  5.  3. nan]
 [nan  3.  4.  1.]
 [ 3.  4. nan  4.]]
